In [ ]:
# basic cherry-pick
import pandas as pd

from src.hamilton_protocols.protocols.cherry import (
    CherryPickPlateParams,
    CherryPickProtocolParams,
    cherry_pick_protocol,
)
from src.hamilton_protocols.utils import dataframe_to_base64

df_b64 = dataframe_to_base64(pd.read_csv("test.csv"))

protocol = cherry_pick_protocol(
    CherryPickProtocolParams(
        plates=[
            CherryPickPlateParams(
                plate_id="P-DIL-0172",
                csv_data=df_b64,
            ),
        ]
    ),
    simulate=True,
)

async for response in protocol.run():
    print(response)

In [ ]:
# fab remapping
import pandas as pd

from hamilton_protocols.protocols.dna import (
    FAbMappingParams,
    FAbMappingProtocolParams,
    fab_mapping_protocol,
)
from hamilton_protocols.utils import dataframe_to_base64

df_b64 = dataframe_to_base64(
    pd.read_csv("C:\\Users\\Adaptyvbio\\Documents\\MAPS\\CRD-002-012_fabs.csv")
)

protocol = fab_mapping_protocol(
    FAbMappingProtocolParams(plates=[FAbMappingParams(plate_id="11", csv_data=df_b64)])
)

async for response in protocol.run():
    pass

In [ ]:
# dna reconstitution
import pandas as pd

from hamilton_protocols.protocols.dna import (
    DNAPlateParams,
    DNAReconstitutionParams,
    dna_reconstitution_protocol,
)
from src.hamilton_protocols.utils import dataframe_to_base64

plates = [191, 187, 190, 188]

plate_data = {}
for plate in plates:
    plate_data[plate] = dataframe_to_base64(
        pd.read_csv(
            f"C:\\Users\\Adaptyvbio\\Documents\\MAPS\\P-TWI-0{plate}_hamilton_reconstitution_mappings.csv"
        )
    )


protocol = dna_reconstitution_protocol(
    DNAReconstitutionParams(
        plates=[
            DNAPlateParams(plate_id=f"{plate}", csv_data=data)
            for plate, data in plate_data.items()
        ]
    ),
    simulate=False,
)

async for response in protocol.run():
    print(response)

In [ ]:
# SPR

from adaptyv_lab import Protocol

protocol = Protocol.from_layout(
    "test",
    "/home/finyl/adaptyv-dev/hamilton-protocols/src/hamilton_protocols/layouts/bli.lay",
    simulator_mode=True,
)
if protocol.deck is None:
    raise ValueError("No deck found in the layout file.")

plate = protocol.deck.get_plate("E5")
reservoir = protocol.deck.get_reservoir("B3")
tips = protocol.deck.get_tip_rack("A3")
src_plate_stack = protocol.deck.get_plate_stack("F4")[:4]
dst_plate_stack = protocol.deck.get_plate_stack("F3")[:4][::-1]

protocol.initialize()

# add buffer
for _ in range(4):
    protocol.pickup_tips(tips)
    protocol.grip_get(src_plate_stack.pop()).grip_place(plate)
    for q in [(0, 1), (1, 0), (1, 1)]:
        for _ in range(2):
            protocol.aspirate(reservoir[10], 40)
            protocol.dispense(plate[q], 40, liquid_height=2)
    protocol.eject_tips(mode=1)
    protocol.pickup_tips(tips[::2, -20::2])
    protocol.aspirate(reservoir, 40)
    protocol.dispense(plate[0, 0], 40, liquid_height=2)
    protocol.eject_tips(mode=1)
    protocol.grip_get(plate).grip_place(dst_plate_stack.pop())

# # add samples
# for q in [(0, 0), (1, 0), (0, 1), (1, 1)]:
#     protocol.grip_get(tip_stack.pop()).grip_place(tips)
#     protocol.grip_get(sample_plate_stack_src.pop()).grip_place(sample_plate)

#     protocol.pickup_tips(tips)
#     protocol.aspirate(sample_plate, 8)
#     protocol.dispense(plate[q], 8, liquid_height=15, mix_cycles=2, mix_volume=10)
#     protocol.eject_tips(mode=2)

#     protocol.grip_get(sample_plate).grip_place(sample_plate_stack_dst.pop())
#     protocol.grip_get(tips).grip_place(tips, waste=True)

protocol.grip_eject()

async for response in protocol.run():
    print(response)

In [ ]:
# dna dilution

from hamilton_protocols.protocols.dna import (
    DNADilutionDestinationPlateParams,
    DNADilutionProtocolParams,
    DNADilutionSourcePlateParams,
    dna_dilution_protocol,
)

protocol = dna_dilution_protocol(
    DNADilutionProtocolParams(
        plates=[
            DNADilutionDestinationPlateParams(
                plate_id="P-DIL-0153",
                source_plates=[
                    DNADilutionSourcePlateParams(
                        plate_id="P-REC-0105",
                        rows=8,
                        cols=6,
                    ),
                ],
            ),
            DNADilutionDestinationPlateParams(
                plate_id="P-DIL-0154",
                source_plates=[
                    DNADilutionSourcePlateParams(
                        plate_id="P-REC-0105",
                        rows=8,
                        cols=3,
                        source_well="A7",
                    ),
                    DNADilutionSourcePlateParams(
                        plate_id="P-REC-0106",
                        rows=3,
                        cols=1,
                    ),
                ],
            ),
        ]
    ),
    simulate=True,
)

In [ ]:
# expression utils

lay_file = "C:\\Users\\Adaptyvbio\\dev\\hamilton-protocols\\src\\hamilton_protocols\\layouts\\reconstitute.lay"

protocol = Protocol.from_layout("test", lay_file, simulator_mode=False)

if protocol.deck is None:
    raise ValueError("No deck found in the layout file.")
print(protocol.deck.grid)

tip_rack = protocol.deck.get_tip_rack("D2")
tip_stack = protocol.deck.get_tip_rack_stack("E3")[:2]
exp_plates_src = protocol.deck.get_plate_stack("F2")[:2]
exp_plates_dst = protocol.deck.get_plate_stack("F3")[:2][::-1]
dil_plates_src = protocol.deck.get_plate_stack("F1")[:2]
dil_plates_dst = protocol.deck.get_plate_stack("F4")[:2][::-1]
exp_plate = protocol.deck.get_plate("C2")
dil_plate = protocol.deck.get_plate("C3")

volumes = [1.5, 1]

protocol.initialize()
for vol in volumes:
    protocol.grip_get(exp_plates_src.pop()).grip_place(exp_plate)
    protocol.grip_get(dil_plates_src.pop()).grip_place(dil_plate)
    protocol.grip_get(tip_stack.pop()).grip_place(tip_rack)

    protocol.pickup_tips(tip_rack)
    protocol.aspirate(dil_plate, vol).dispense(
        exp_plate, vol, mix_cycles=3, mix_volume=2
    )
    protocol.eject_tips(mode=2)

    protocol.grip_get(exp_plate).grip_place(exp_plates_dst.pop())
    protocol.grip_get(dil_plate).grip_place(dil_plates_dst.pop())
    protocol.grip_get(tip_rack).grip_place(tip_rack, waste=True)
protocol.grip_eject()

async for response in protocol.run():
    print(response)

In [ ]:
from src.hamilton_protocols.api.log_analyzer import (
    analyze_all_logs,
    get_combined_analysis,
)

log_dir = "C:\\Program Files (x86)\\HAMILTON\\LogFiles"

logs = analyze_all_logs(log_dir)
stats = get_combined_analysis(logs)
print(stats)